In [1]:
%matplotlib inline
import numpy as np
import lib.io.stan
import lib.plots.stan
import matplotlib.pyplot as plt
import os
from matplotlib.lines import Line2D
import retro_prepare
import matplotlib.colors
import mne
import pandas as pd
import re
import glob

In [2]:
import json
with open(f'../ANSM/util/data/ei-vep.json','r') as f:
        ezh_all = json.load(f)

In [3]:
pidlist=[pid for pid in ezh_all.keys()]

In [4]:
pidlist

['id001_bt',
 'id003_mg',
 'id004_bj',
 'id007_rd',
 'id010_cmn',
 'id013_lk',
 'id014_vc',
 'id017_mk',
 'id020_lma',
 'id021_jc',
 'id022_te',
 'id023_br',
 'id025_mc',
 'id027_sj',
 'id028_ca',
 'id030_bf',
 'id033_fc',
 'id036_dm',
 'id037_cg',
 'id039_mra',
 'id040_ms',
 'id050_sx',
 'id008_dmc']

In [7]:
raw_fnames

['/home/hfw/Retro/datasets/id003_mg/seeg/fif/MGcrise1.bip.raw.fif',
 '/home/hfw/Retro/datasets/id003_mg/seeg/fif/MGcrise3_0000.bip.raw.fif',
 '/home/hfw/Retro/datasets/id003_mg/seeg/fif/MGcriseP2+G.bip.raw.fif']

In [6]:
for pid in ['id003_mg']:#pidlist[2:3]:
    pid_dir = f'/home/hfw/Retro/datasets/{pid}'

    print(pid_dir)
    raw_fnames = glob.glob(f'{pid_dir}/seeg/fif/*.bip.raw.fif')
    data_dir = f'/home/hfw/Retro/datasets/{pid}'

    results_dir = f'/home/hfw/Retro/results/{pid}'


    nchan=162


    #ezh=np.zeros([nchan,1])

    #ezh[ezh_all[pid]['i_ez']]=1


    # In[14]:

    for raw_fname in raw_fnames:
        print(raw_fname)
        basicfilename=raw_fname.split('.bip')[0]
        szr_name =basicfilename.split('fif/')[1]
        meta_data_fname = f'{szr_name}.json'
        #raw_seeg_fname = f'{szr_name}.raw.fif'
        fname_suffix = f'{szr_name}'
        os.makedirs(results_dir,exist_ok=True)
        os.makedirs(f'{results_dir}/logs',exist_ok=True)
        os.makedirs(f'{results_dir}/figures',exist_ok=True)
        os.makedirs(f'{results_dir}/Rfiles', exist_ok=True)
        hpf = 10
        lpf = 0.02
        raw_seeg_fname = f'{szr_name}.raw.fif'
        data = retro_prepare.prepare_data_bip(data_dir, meta_data_fname, raw_seeg_fname, szr_name, hpf, lpf)
        seeg, bip = retro_prepare.read_one_seeg(data_dir, meta_data_fname, raw_seeg_fname)
        slp = retro_prepare.compute_slp(seeg, bip, hpf, lpf)
        all_fb_d0=np.load(f'{results_dir}/EZdelay/ez_prior_{szr_name}.npy')

        d0_prior=np.mean(all_fb_d0,axis=0)
        ez_prior=np.where(d0_prior>0.5)

        ds_freq = int(data['slp'].shape[0]/150)
        data['slp'] = data['slp'][0:-1:ds_freq]
        data['snsr_pwr'] = (data['slp']**2).mean(axis=0)
        data['ns'], data['nn'] = data['gain'].shape
        data['nt'] = data['slp'].shape[0]
        ez_hyp = np.where(np.loadtxt(f'{data_dir}/tvb/ez_hypothesis.vep.txt') == 1)[0]
        data['x0_mu'] = -3.0*np.ones(data['nn'])
        fname_suffix += f'_hpf{hpf}_lpf{lpf}_ezdelay'
        data['x0_mu'][ez_prior[0]] = -1.5
        stan_fname = 'szr_prpgtn'

        x0 = data['x0_mu']
        amplitude = 1.0 
        offset = 0
        K = 1.0
        tau0 = 20
        eps_slp = 1.0
        eps_snsr_pwr = 1.0
        x_init = -2.0*np.ones(data['nn'])
        z_init = 3.5*np.ones(data['nn'])

        param_init = {'x0':x0, 'amplitude':amplitude,
              'offset':offset, 'K':K, 'tau0':tau0, 'x_init':x_init, 'z_init':z_init,
              'eps_slp':eps_slp, 'eps_snsr_pwr':eps_snsr_pwr}

        param_init_file = 'param_init.R'
        os.makedirs(f'{results_dir}/Rfiles',exist_ok=True)
        lib.io.stan.rdump(f'{results_dir}/Rfiles/param_init.R',param_init)

        input_Rfile = f'fit_data_{fname_suffix}.R'
        os.makedirs(f'{results_dir}/Rfiles',exist_ok=True)
        lib.io.stan.rdump(f'{results_dir}/Rfiles/{input_Rfile}',data)


/home/hfw/Retro/datasets/id003_mg
/home/hfw/Retro/datasets/id003_mg/seeg/fif/MGcrise1.bip.raw.fif
Isotrak not found
Isotrak not found
/home/hfw/Retro/datasets/id003_mg/seeg/fif/MGcrise3_0000.bip.raw.fif
Isotrak not found


ValueError: The length of the input vector x must be at least padlen, which is 12.

In [198]:
hpf = 10
lpf = 0.02
pid = pidlist[22]
print(pid)
pid_dir = f'/home/hfw/Retro/datasets/{pid}'
raw_fnames = glob.glob(f'{pid_dir}/seeg/fif/*.bip.raw.fif')
print(len(raw_fnames))
raw_fname=raw_fnames[3]
basicfilename=raw_fname.split('.bip')[0]
szr_name =basicfilename.split('fif/')[1]
        
stan_fname = 'szr_prpgtn'
results_dir = f'/home/hfw/Retro/results/{pid}'
fname_suffix = f'{szr_name}_hpf{hpf}_lpf{lpf}_ezdelay'
input_Rfile = f'fit_data_{fname_suffix}.R'
print(szr_name)






id008_dmc
4
DMC_criseGS_161111M-AEX_0007


In [199]:
%%bash -s "$stan_fname" "$results_dir" "$input_Rfile" "$fname_suffix"

STAN_FNAME=$1
RESULTS_DIR=$2
INPUT_RFILE=$3
FNAME_SUFFIX=$4

for i in {1..2};
do
./${STAN_FNAME} optimize algorithm=lbfgs tol_param=1e-4 iter=20000 save_iterations=0  \
data file=${RESULTS_DIR}/Rfiles/${INPUT_RFILE} \
init=${RESULTS_DIR}/Rfiles/param_init.R \
output file=${RESULTS_DIR}/samples_${FNAME_SUFFIX}_chain${i}.csv refresh=10 \
&> ${RESULTS_DIR}/logs/snsrfit_ode_${FNAME_SUFFIX}_chain${i}.log &
done

In [9]:
#stan_fname = 'vep-snsrfit-ode-nointerp'


In [10]:
#%%bash -s "$stan_fname" 

#stancompile.sh $1